In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import os

%matplotlib inline

path = '/'.join(os.getcwd().split('/')[:-1])
print(path)

tci = pd.read_csv(path+'/data/model_data/tci_2_1.csv', parse_dates=['Date'])

ppns = set(tci['parcel'])
dates = dict(zip(tci.parcel, tci.Date))

print(tci.shape)

/Volumes/Dropbox/largetransfer/luc/carter
(113132, 43)


## Violations

In [2]:
vs = pd.read_csv(path+'/data/clean_data/violations.csv', parse_dates = [1,4])
vs = vs.loc[vs.apply(lambda x: x.v_file_date < dates[x.parcel], axis=1)]

In [3]:
vs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166685 entries, 0 to 170837
Data columns (total 18 columns):
VIN                     166685 non-null object
v_file_date             166685 non-null datetime64[ns]
v_wf_task               166685 non-null object
v_wf_task_status        164852 non-null object
v_wf_task_date          166685 non-null datetime64[ns]
v_type_of_violation     160269 non-null object
parcel1                 122437 non-null object
parcel                  166685 non-null object
cond_gar                14894 non-null float64
cond                    40680 non-null float64
other                   26644 non-null float64
maint                   71716 non-null float64
fire                    4763 non-null float64
int_ext                 1540 non-null float64
vin_id                  123728 non-null object
violation_issue_date    123728 non-null object
vn_created              123728 non-null float64
vn_source               123728 non-null float64
dtypes: datetime64[ns](2), 

### Violation Task

In [4]:
# vs_task = pd.DataFrame(vs.groupby('v_wf_task').count()['VIN'])
# vs_task.columns = ['violations_tasks']
# vs_task[vs_task.violations_tasks > 1]

In [5]:
# for task in set(vs.v_wf_task):
#     tci = pd.merge(tci, vs.loc[vs.v_wf_task==task,['parcel','v_wf_task']].groupby('parcel').count(), how='left', left_on = 'ppn', right_index=True)
#     tci = tci.rename(columns={'v_wf_task': task.lower()+' count'})
#     tci[task.lower()+' count'] = tci[task.lower()+' count'].fillna(0)
# #     print task, len(a), sum(a.vacant)*1.0/len(a.vacant)

In [6]:
# def check_task(x):
#     if x in parcel_set:
#         return 1
#     else:
#         return 0
    
# for task in vs_task.index:
#     parcel_set = set(vs.loc[vs.v_wf_task==task,'parcel'])
#     tci['task_'+task[0:3].lower()] = tci['ppn'].apply(check_task)

### Violations count 

In [7]:
vs_copy = vs.groupby('VIN').last().reset_index()

vs_copy = vs_copy[['parcel','v_file_date']].groupby('parcel').count().reset_index()

vs_copy['vs_count'] = vs_copy.apply(lambda x: \
        len(vs[(vs.parcel == x.parcel) & (vs.v_file_date > dates[x.parcel]-pd.DateOffset(years=1))]), axis=1)

vs_copy = vs_copy.rename(columns={'vs_count':'v_total_1yr'})
vs_copy = vs_copy.set_index('parcel')

In [8]:
tci = pd.merge(tci, vs_copy[['v_total_1yr']], \
               how='left', left_on='parcel', right_index=True)
tci.loc[tci.v_total_1yr.isnull(), 'v_total_1yr'] = 0

In [9]:
vs_copy = vs.groupby('VIN').last().reset_index()

vs_copy = vs_copy[['parcel','v_file_date']].groupby('parcel').count().reset_index()

vs_copy['vs_count'] = vs_copy.apply(lambda x: \
        len(vs[(vs.parcel == x.parcel) & (vs.v_file_date > dates[x.parcel]-pd.DateOffset(years=2))]), axis=1)

vs_copy = vs_copy.rename(columns={'vs_count':'v_total_2yr'})
vs_copy = vs_copy.set_index('parcel')

In [10]:
tci = pd.merge(tci, vs_copy[['v_total_2yr']], \
               how='left', left_on='parcel', right_index=True)
tci.loc[tci.v_total_2yr.isnull(), 'v_total_2yr'] = 0

In [11]:
# vs_count = pd.DataFrame(vs[vs['v_file_date']>dt.datetime(2013,6,1)].groupby('VIN').last().groupby('parcel').count()['v_file_date'])
# vs_count = vs_count.rename(columns={'v_file_date':'violations_number_2_yr'})
# tci = pd.merge(tci, vs_count[['violations_number_2_yr']], \
#                how='left', left_on='ppn', right_index=True)
# tci.loc[tci.violations_number_2_yr.isnull(), 'violations_number_2_yr'] = 0

### Violation types

In [12]:
# vs_type = vs.loc[vs['v_file_date']>dt.datetime(2013,6,1),['v_type_of_violation','parcel','VIN']].groupby('VIN').last().groupby('v_type_of_violation').count()
# vs_type = vs_type.rename(columns={'parcel':'violations_type'})

In [13]:
# violation_types = vs_type[vs_type['violations_type']>500]

In [14]:
# for vio in violation_types.index:
#     tem = vs.loc[(vs['v_file_date']>dt.datetime(2013,6,1)) & (vs.v_type_of_violation==vio),['v_type_of_violation','parcel','VIN']].groupby('VIN').last().groupby('parcel').count()
#     tci = pd.merge(tci, tem, left_on='ppn', right_index=True, how='left')
#     tci = tci.rename(columns={'v_type_of_violation': 'v_'+vio.lower()[0:6]+'_count'})
#     tci['v_'+vio.lower()[0:6]+'_count'] = tci['v_'+vio.lower()[0:6]+'_count'].fillna(0)

In [15]:
tci.columns

Index(['parcel', 'vacant', 'Date', 'Survey Category', 'Survey Grade',
       'propsize', 'totusabl', 'tmktval', 'condition13', 'condition14',
       'condition_value13', 'condition_value14', 'style_filtered',
       'ownerocc_value', 'totbldgs', 'yrbuilt_filtered', 'rextwall',
       'vacant_block', 'median_rent', 'property_crimes', 'burglaries',
       'part_one_crimes', 'part_two_crimes', 'bachelors+', 'poverty_rate',
       'median_hh_income', 'white', 'black', 'asian', 'other', 'hispanic',
       'young', 'middle', 'old', 'al_num', 'al_date', 'al_days', 't_days',
       't_shf_deed', 't_num', 'lb_acquired', 'lb_tax_fc', 'pv_total',
       'v_total_1yr', 'v_total_2yr'],
      dtype='object')

## Complaints

In [16]:
cp = pd.read_csv(path+'/data/clean_data/complaints.csv', parse_dates = [2],index_col=[0], dtype=str)
cp = cp.loc[cp.apply(lambda x: x.c_file_date < dates[x.parcel], axis=1)]
cp = cp.sort_values('c_file_date')

In [17]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260807 entries, 29563 to 466314
Data columns (total 20 columns):
complaint_id           260807 non-null object
c_file_date            260807 non-null datetime64[ns]
c_wf_task              260807 non-null object
c_wf_task_status       247683 non-null object
c_wf_task_date         260807 non-null object
c_type_of_complaint    256052 non-null object
cdc_ri                 65 non-null object
cdc_corrected          177 non-null object
cdc_referred           486 non-null object
othercomp              156810 non-null object
parcel                 260807 non-null object
parcel1                81757 non-null object
cdc_comp               0 non-null object
cdccia_comp            0 non-null object
council_comp           0 non-null object
mac_comp               0 non-null object
public_comp            0 non-null object
other_comp             0 non-null object
complaint_id1          226633 non-null object
comp_source            254713 non-null objec

In [18]:
cp.head(2)

,complaint_id,c_file_date,c_wf_task,c_wf_task_status,c_wf_task_date,c_type_of_complaint,cdc_ri,cdc_corrected,cdc_referred,othercomp,parcel,parcel1,cdc_comp,cdccia_comp,council_comp,mac_comp,public_comp,other_comp,complaint_id1,comp_source
29563,CMP05007059-M,2005-10-28,Inspection,Inspection Denied,05/28/2009,NaN,NaN,NaN,NaN,1,111-22-121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CMP05007059-M,NaN
29564,CMP05007059-M,2005-10-28,Closure,Closed-ADMIN,05/28/2009,NaN,NaN,NaN,NaN,1,111-22-121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CMP05007059-M,NaN


In [19]:
# cp.groupby('complaint_id').last().groupby(.agg(lambda x: len(set(x.parcel)))

In [20]:
types = cp[cp.c_file_date > dt.datetime(2013,6,1)].groupby('complaint_id').last().groupby('c_type_of_complaint').count()['c_file_date'].sort_values(ascending=False).head()
types

c_type_of_complaint
O.V.V.                          9164
Complete interior/exterior      2892
General Exterior Maintenance    2346
Fire Damage                      976
Vacant Distressed                695
Name: c_file_date, dtype: int64

In [21]:
# types = ['Black Mold','Collapsing Structure','Complete interior/exterior', \
#          'Debris/Garbage/Junk (Occupied)','Electrical','Fire Damage', \
#          'General Exterior Maintenance','No Permit','O.V.V.', 'Plumbing']

### Complaint type

In [22]:
cp_copy = cp[cp.c_type_of_complaint=='O.V.V.'].groupby('complaint_id').last().reset_index()

cp_copy = cp_copy[['parcel','c_file_date']].groupby('parcel').count().reset_index()

cp_copy['c_ovv_2yr'] = cp_copy.apply(lambda x: \
        len(cp[(cp.parcel == x.parcel) & (cp.c_file_date > dates[x.parcel]-pd.DateOffset(years=2))]), axis=1)

cp_copy = cp_copy.set_index('parcel')

tci = pd.merge(tci, cp_copy[['c_ovv_2yr']], \
               how='left', left_on='parcel', right_index=True)
tci.loc[tci.c_ovv_2yr.isnull(), 'c_ovv_2yr'] = 0

In [23]:
print(np.mean(tci[tci.vacant==1].c_ovv_2yr))
print(np.mean(tci[tci.vacant==0].c_ovv_2yr))

3.9696154612472605
0.1616000931189012


In [24]:
cp_copy = cp[cp.c_type_of_complaint=='O.V.V.'].groupby('complaint_id').last().reset_index()

cp_copy = cp_copy[['parcel','c_file_date']].groupby('parcel').count().reset_index()

cp_copy['c_ovv_1yr'] = cp_copy.apply(lambda x: \
        len(cp[(cp.parcel == x.parcel) & (cp.c_file_date > dates[x.parcel]-pd.DateOffset(years=1))]), axis=1)

cp_copy = cp_copy.set_index('parcel')

tci = pd.merge(tci, cp_copy[['c_ovv_1yr']], \
               how='left', left_on='parcel', right_index=True)
tci.loc[tci.c_ovv_1yr.isnull(), 'c_ovv_1yr'] = 0

In [25]:
print(np.mean(tci[tci.vacant==1].c_ovv_1yr))
print(np.mean(tci[tci.vacant==0].c_ovv_1yr))

1.9460051803148037
0.07472791821056511


In [26]:
# for complaint in ['O.V.V.']:
#     t = cp.loc[(cp.c_file_date > dt.datetime(2015,1,1)) & (cp.c_type_of_complaint==complaint),['c_type_of_complaint','parcel','complaint_id']] \
#             .groupby('complaint_id').last().groupby('parcel').count()[['c_type_of_complaint']]
#     t = t.rename(columns={'c_type_of_complaint': 'c_'+complaint+'_0yr'})
#     tem = pd.merge(tci, t, left_on='ppn', right_index=True, how='left')
#     tem['c_'+complaint+'_0yr'] = tem['c_'+complaint+'_0yr'].fillna(0)
#     print pd.crosstab(tem['c_'+complaint+'_0yr'], tem.vacant)

In [27]:
# for complaint in types.index:
#     t = cp.loc[(cp.c_file_date > dt.datetime(2014,6,1)) & (cp.c_type_of_complaint==complaint),['c_type_of_complaint','parcel','complaint_id']] \
#             .groupby('complaint_id').last().groupby('parcel').count()[['c_type_of_complaint']]
#     t = t.rename(columns={'c_type_of_complaint': 'c_'+complaint+'_1yr'})
#     tci = pd.merge(tci, t, left_on='ppn', right_index=True, how='left')
#     tci['c_'+complaint+'_1yr'] = tci['c_'+complaint+'_1yr'].fillna(0)
# #     print pd.crosstab(tem['c_'+complaint+'_1yr'], tem.vacant)

In [28]:
# for complaint in types.index:
#     t = cp.loc[(cp.c_file_date > dt.datetime(2013,6,1)) & (cp.c_type_of_complaint==complaint),['c_type_of_complaint','parcel','complaint_id']] \
#             .groupby('complaint_id').last().groupby('parcel').count()[['c_type_of_complaint']]
# #     t = cp.loc[cp.c_type_of_complaint==complaint,['c_type_of_complaint','parcel']].groupby('parcel').count()
#     t = t.rename(columns={'c_type_of_complaint': 'c_'+complaint+'_2yr'})
#     tci = pd.merge(tci, t, left_on='ppn', right_index=True, how='left')
#     tci['c_'+complaint+'_2yr'] = tci['c_'+complaint+'_2yr'].fillna(0)
# #     print pd.crosstab(tem['c_'+complaint+'_2yr'], tem.vacant)

### Complaint count (2 yrs)

In [29]:
# cp_count = pd.DataFrame(cp[(cp.c_file_date > dt.datetime(2013,6,1))].groupby('parcel').count()['complaint_id'])
# cp_count.columns = ['complaint_count_2yr']
# tci = pd.merge(tci, cp_count, \
#                how='left', left_on='ppn', right_index=True)
# tci.loc[tci.complaint_count_2yr.isnull(), 'complaint_count_2yr'] = 0

In [30]:
# sum(tci.loc[tci.complaint_count_2yr>5, 'vacant'])*1.0/len(tci.loc[tci.complaint_count_2yr>5, 'vacant'])

### Complaint count (1 yr)

In [31]:
# cp_count = pd.DataFrame(cp[cp.c_file_date>dt.datetime(2014, 6, 1)].groupby('parcel').count()['complaint_id'])
# cp_count.columns = ['complaint_count_1yr']
# tci = pd.merge(tci, cp_count, \
#                how='left', left_on='ppn', right_index=True)
# tci.loc[tci.complaint_count_1yr.isnull(), 'complaint_count_1yr'] = 0

In [32]:
# sum(tci.loc[tci.complaint_count_1yr>2, 'vacant'])*1.0/len(tci.loc[tci.complaint_count_1yr>2, 'vacant'])

### Complain count (6 months)

In [33]:
# cp_count = pd.DataFrame(cp[cp.c_file_date>dt.datetime(2015, 1, 1)].groupby('parcel').count()['complaint_id'])
# cp_count.columns = ['complaint_count_6_mon']
# tci = pd.merge(tci, cp_count, \
#                how='left', left_on='ppn', right_index=True)
# tci.loc[tci.complaint_count_6_mon.isnull(), 'complaint_count_6_mon'] = 0

In [34]:
# sum(tci.loc[tci.complaint_count_6_mon>0, 'vacant'])*1.0/len(tci.loc[tci.complaint_count_6_mon>0, 'vacant']) ,len(tci.loc[tci.complaint_count_6_mon>0, 'vacant'])

In [35]:
tci.to_csv(path+'/data/model_data/tci_2_2.csv', index=False)

In [36]:
tci.shape

(113132, 47)

In [37]:
tci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113132 entries, 0 to 113131
Data columns (total 47 columns):
parcel               113132 non-null object
vacant               113132 non-null int64
Date                 113132 non-null datetime64[ns]
Survey Category      113132 non-null object
Survey Grade         113132 non-null object
propsize             113132 non-null float64
totusabl             113132 non-null float64
tmktval              113132 non-null float64
condition13          112559 non-null object
condition14          112595 non-null object
condition_value13    113132 non-null float64
condition_value14    113132 non-null float64
style_filtered       113132 non-null object
ownerocc_value       113132 non-null float64
totbldgs             113132 non-null float64
yrbuilt_filtered     113132 non-null float64
rextwall             113132 non-null object
vacant_block         113132 non-null float64
median_rent          113132 non-null float64
property_crimes      113132 non-null